In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
df = pd.read_csv("train.csv", index_col="id")

In [ ]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
df["toxicity"] = df["toxic"] + df["severe_toxic"] + df["obscene"] + df["threat"] + df["insult"] + df["identity_hate"]
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxicity
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [ ]:
df["toxicity"].value_counts()

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
Name: toxicity, dtype: int64

In [ ]:
data = df.iloc[:,0].reset_index()

In [ ]:
data.pop("id")
data.head()

,comment_text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ..."
4,"You, sir, are my hero. Any chance you remember..."


In [ ]:
target = df.iloc[:,-1].reset_index()

In [ ]:
target.pop("id")
target

,toxicity
0,0
1,0
2,0
3,0
4,0
...,...
159566,0
159567,0
159568,0
159569,0


In [ ]:
from sklearn.model_selection import train_test_split
dftrain, dftest, dftrain_labels, dftest_labels = train_test_split(data, target, test_size = 0.2)

In [ ]:
dftrain

,comment_text
133355,"""\n\n@Mbshick: It sure looks like it doesn't b..."
156040,Nigga Owe Me Some Money \n\nYoung Buck And Pli...
92758,"""\n\n In the criticism and research section: '..."
70625,"""\nRE: To be clear: I reacted seriously and ex..."
89626,"""\nIf you leave a message on someone's talk pa..."
...,...
130108,supplied to Soviet army air force teams on the...
92523,Apologies \n\nI'm taking the first oppourtunit...
28515,"Jotamar, I think that your arguments have beco..."
622,"""\n\nI think the entire article should be tagg..."


In [ ]:
dftrain_labels

,toxicity
133355,0
156040,2
92758,0
70625,0
89626,0
...,...
130108,0
92523,0
28515,0
622,0


In [ ]:
dftest

,comment_text
7838,My twopenn'orth is that it is probably better ...
77812,Image:Osx-in-action-small.gif listed for delet...
94235,"""Welcome!\n\nHello, Jaxl, and welcome to Wikip..."
6837,"""\nOdd, then, that multiple images of Batman a..."
56543,"Sorry \n\nSorry, I was still editing that when..."
...,...
25470,"Ok, I will leave it on for now for another use..."
151560,are you stupid? \n\nwill you fucking stop remo...
9144,The last two episode of Jon & Kate Plus 8 have...
80907,"WAT \n\nOoooh, I'll be blocked! IMAGINE THAT!!..."


In [ ]:
dftest_labels

,toxicity
7838,0
77812,0
94235,0
6837,0
56543,0
...,...
25470,0
151560,3
9144,0
80907,0


In [ ]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
    global word_encoding

    words = text.lower().split(" ") 
    encoding = []  

    for word in words:
        if word in vocab:
            code = vocab[word]  
            encoding.append(code) 
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            word_encoding += 1
    return encoding

In [ ]:
dftrain["encoding"] = [one_hot_encoding(dftrain.iloc[i,0]) for i in range(len(dftrain))]

In [ ]:
dftest["encoding"] = [one_hot_encoding(dftrain.iloc[i,0]) for i in range(len(dftest))]

In [ ]:
dftrainnew = dftrain["encoding"]
dftestnew = dftest["encoding"]

In [ ]:
dftrain_labels = dftrain_labels["toxicity"]
dftest_labels = dftest_labels["toxicity"]

In [ ]:
ar = []
for i in dftrain_labels.values:
    ar.append(i)
dftrain_labels_encoding = np.array(ar)

ar1 = []
for i in dftest_labels.values:
    ar1.append(i)
dftest_labels_encoding = np.array(ar1)

In [ ]:
dftrainnew_encoding = np.asarray(dftrainnew.values.tolist())
dftestnew_encoding = np.asarray(dftestnew.values.tolist())

In [ ]:
dftrainnew_encoding

array([list([1, 2, 3, 4, 5, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
       list([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]),
       list([34, 35, 36, 37, 28, 38, 39, 40, 41, 42, 43, 36, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 54, 56, 57, 58, 59, 60, 61, 47, 62, 30, 45, 63, 64, 65, 66, 67, 54, 36, 68, 31, 69, 70, 71, 36, 72, 54, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 54, 86, 87, 35, 88, 38, 89, 90, 91, 77, 78, 79, 80, 92, 93, 94, 37, 35, 95, 86, 87, 35, 88, 38, 96, 90, 97, 77, 78, 79, 80, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 35, 36, 110, 111, 112, 113, 114, 38, 115, 36, 72, 54, 116, 117, 77, 118, 119, 35, 120, 47, 121, 122, 123, 124, 32, 125, 126, 30, 127, 128, 129, 130, 36, 131, 54, 36, 132, 47, 133, 134, 135, 45, 136, 85, 54, 137, 35, 95, 138, 139, 140, 141, 54, 36, 142, 54, 36, 143, 14, 144, 145, 28, 146, 147, 148, 149, 150, 151, 152, 152, 20]),
       ...,
       list([465229, 8, 402, 32, 685, 519, 42, 1541, 57

In [ ]:
dftrain_labels_encoding

array([0, 2, 0, ..., 0, 0, 0])

In [ ]:
dftrainnew_encoding = tf.keras.preprocessing.sequence.pad_sequences(dftrainnew_encoding, 250)
dftestnew_encoding = tf.keras.preprocessing.sequence.pad_sequences(dftestnew_encoding, 250)

In [ ]:
dftrainlabels = tf.one_hot(dftrain_labels_encoding, 6)
dftestlabels = tf.one_hot(dftest_labels_encoding, 6)

In [ ]:
VOCAB_SIZE = 355628
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(6, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         35562800  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 35,680,822
Trainable params: 35,680,822
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])

In [ ]:
history = model.fit( tf.convert_to_tensor(dftrainnew_encoding),  tf.convert_to_tensor(dftrainlabels), batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
798/798 [==============================] - 32s 40ms/step - loss: 0.0441 - acc: 0.9847 - val_loss: 0.3982 - val_acc: 0.9021
Epoch 2/10
798/798 [==============================] - 31s 39ms/step - loss: 0.0281 - acc: 0.9909 - val_loss: 0.4773 - val_acc: 0.9018
Epoch 3/10
798/798 [==============================] - 31s 39ms/step - loss: 0.0198 - acc: 0.9935 - val_loss: 0.5001 - val_acc: 0.9044
Epoch 4/10
798/798 [==============================] - 31s 39ms/step - loss: 0.0151 - acc: 0.9950 - val_loss: 0.5378 - val_acc: 0.9035
Epoch 5/10
798/798 [==============================] - 31s 39ms/step - loss: 0.0111 - acc: 0.9964 - val_loss: 0.5590 - val_acc: 0.8945
Epoch 6/10
798/798 [==============================] - 32s 40ms/step - loss: 0.0087 - acc: 0.9972 - val_loss: 0.5775 - val_acc: 0.9024
Epoch 7/10
798/798 [==============================] - 31s 39ms/step - loss: 0.0072 - acc: 0.9975 - val_loss: 0.5973 - val_acc: 0.9000
Epoch 8/10
798/798 [==============================] - 31s 39ms

In [44]:
results = model.evaluate(dftestnew_encoding, dftestlabels)
print(results)

998/998 [==============================] - 7s 7ms/step - loss: 2.5999 - acc: 0.8112
[2.5998663902282715, 0.8111859560012817]


In [46]:
def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [vocab[word] if word in vocab else 0 for word in tokens]
  return tf.keras.preprocessing.sequence.pad_sequences([tokens], 250)[0]
encode_text("Fuck you bitch")

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [62]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  return list(result[0]).index(max(list(result[0])))
predict("My grandma telling me that I have to put up with nasty family members “because they’re FaaaMilYYYYYyyyy”. Gtfo, they are bad people and I’m not going to let them poison my life because we’re related.")

2